<a href="https://colab.research.google.com/github/kylejgillett/sounderpy/blob/main/FIELD_CAMPAIGN_SOUNDING_PLOTTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIELD CAMPAIGN SOUNDING VIEWER
### POWERED BY SOUNDERPY

- *Questions? Contact Kyle Gillett (@wxkylegillett, kyle.gillett@und.edu)*
- *SounderPy Documentation: https://kylejgillett.github.io/sounderpy/*
- *Real-time Sounding Analysis Webapp: https://sounderpysoundings.anvil.app/*
- *What is 'Google Colabs'? https://research.google.com/colaboratory/faq.html*

In [ ]:
# PART 1a: INSTALL PACKAGES ONTO VIRTUAL MACHINE (must be done on start up, and may take a moment)
# PART 1b: ON THE LEFT, CLICK THE FILE-FOLDER ICON, CLICK ON THE FILE-UPLOAD ICON, UPLOAD A **SHARPPY** DATA TXT FILE

!pip install sounderpy
!pip install metpy

In [ ]:
# PART 2: IMPORT REQUIRED PACKAGES

import sounderpy as spy
import numpy as np
import pandas as pd
from metpy.units import units
import metpy.calc as mpcalc

In [ ]:
# PART 3: IMPORT AND PARSE THE ***SHARPPY*** DATA TXT FILE

# load data with pandas, be sure to change the name of the file below
raw_data = pd.read_csv('your-sharppy-file.txt', skiprows=7, header=None)

# drop the last row of SHARPPY %END% vals
raw_data = raw_data.drop(raw_data.index[-1])

# show raw data
raw_data

In [ ]:
# PART 4: BUILD SOUNDERPY DICTIONARY WITH YOUR DATA (format MUST be maintained)

# fill in the variables below
campaign_name  = 'SOME FANCY NAME FOR MY CAMPAIGN'
valid_date     = '20130531' #YYYYMMDD
valid_time     = '1845' #HHMM
site_name      = 'EL RENO, OK'
site_elevation = 352 #in meters, (easily found on Google if not known)
site_latlon    = [41.478,-95.337]


# this adds raw_data to a sounderpy dict
clean_data = {}

clean_data['p']  = np.char.strip(np.array(raw_data[0][0::4].to_list())).astype(float)*units.hPa
clean_data['z']  = np.array(raw_data[1][0::4])*units.m
clean_data['T']  = np.array(raw_data[2][0::4])*units.degC
clean_data['Td'] = np.array(raw_data[3][0::4])*units.degC
clean_data['u'], clean_data['v'] = mpcalc.wind_components(np.array(raw_data[5][0::4])*units('kts'),
                                                          np.array(raw_data[4][0::4])*units.deg)
clean_data['site_info'] = {
            'site-id'   : '   ',                                      # could be a station, site, launch ID, or blank
            'site-name' : site_name,                                  # a site, station, launch or campaign name
            'site-lctn' : 'none',                                     # could be another name, or none
            'site-latlon' : site_latlon,                              # launch lat/lon
            'site-elv'  : site_elevation,                             # the profile's elevation
            'source'    : campaign_name,                              # the 'source' which will be the main title component of the plot
            'model'     : 'none',                                     # model name if a model was involved
            'fcst-hour' : f'none',                                    # forecast hour if a model was involved
            'run-time'  : ['none', 'none', 'none', 'none'],           # model run date if a model was involved
            'valid-time': [valid_date[0:4], valid_date[4:6], valid_date[6:8], valid_time]}                  # the profile's valid date/time.


In [ ]:
# STEP 5: BUILD THE SOUNDING

# optional - set a custom SM based on radar, etc
# recall that Bunkers SM requires 6000m of data to find a Bunkers SM
# you may leave this as 'right_moving', or set [direction, speed]
storm_motion = 'right_moving'
#storm_motion = [230, 40]

spy.build_sounding(clean_data, dark_mode=False, color_blind=True,
                   special_parcels='none', storm_motion=storm_motion)